# 2. **Knowledge Graph**

In [75]:
import pandas as pd
from tqdm import tqdm
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.tools import BaseTool
import yfinance as yf
from dotenv import load_dotenv
import os
from typing import Any

In [76]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["NEO4J_URI"] = "neo4j+s://53d608e5.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "NSdlyOMl34oT6HilZL1wRmBP4nQde0lWopDwotBl9Q0"
graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
)

In [77]:
stocks_complete = pd.read_csv("../data/stocks_completed.csv")
rolling_corr_matrices = pd.read_csv("../data/rolling_corr_matrices.csv")
sector_means = pd.read_csv("../data/sector_means.csv")
sector_volatility = pd.read_csv("../data/sector_volatility.csv")
sector_returns = pd.read_csv("../data/sector_returns.csv")
rolling_sector_corr = pd.read_csv("../data/rolling_sector_corr.csv")

In [78]:
#If we reset clean will affect the langchain.
#graph.run("MATCH (n) DETACH DELETE n")
#print("Cleared the Neo4j database.")

Cleared the Neo4j database.


In [79]:
stocks_complete.columns

Index(['Ticker', 'Name', 'Sector', 'marketCap', 'trailingPE', 'forwardPE',
       'priceToBook', 'trailingEps', 'forwardEps', 'bookValue', 'payoutRatio',
       'beta', 'fiveYearAvgDividendYield', '52WeekChange', 'averageVolume',
       'enterpriseToRevenue', 'profitMargins', 'Close', 'Volume', 'NormClose',
       'DailyLogReturn', 'ALR1M', 'ALR3M', 'Volatility', 'RSI', 'MACD',
       'HighLowRange'],
      dtype='object')

In [80]:
system_prompt = """
You are a financial AI assistant capable of integrating Neo4j graph insights with real-time financial data. 
You can:
1. Retrieve data from Neo4j (e.g., relationships between stocks and sectors).
2. Update the graph with real-time financial data from APIs like yfinance.
3. Answer complex financial queries by combining historical and real-time data.
"""

Adding Sector Nodes: 100%|██████████| 11/11 [00:00<00:00, 34.75it/s]

Total Sector Nodes: 11
Sample Sector Nodes: [{'s.name': 'Information Technology'}, {'s.name': 'Health Care'}, {'s.name': 'Financials'}, {'s.name': 'Real Estate'}, {'s.name': 'Consumer Discretionary'}]


In [81]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

Adding Stock Nodes: 119500it [03:13, 617.01it/s]


In [82]:
class Neo4jGraphTool(BaseTool):
    name: str = "neo4j_graph"  # Name of the tool
    description: str = "A tool for querying and updating the Neo4j graph."  # Description of the tool
    graph: Any  # Declare `graph` as a field

    def __init__(self, graph: Any):
        super().__init__(graph=graph)

    def _run(self, query: str) -> str:
        """
        Execute a Cypher query on the Neo4j graph and return the result.
        """
        try:
            result = self.graph.query(query)
            return str(result)
        except Exception as e:
            return f"Error executing query: {str(e)}"

    async def _arun(self, query: str) -> str:
        """Asynchronous version (required by LangChain)."""
        raise NotImplementedError("Async not supported for this tool.")

Index(['TimeIndex', 'Ticker1', 'Ticker2', 'Correlation'], dtype='object')

In [83]:
neo4j_tool = Neo4jGraphTool(graph=graph)

Preparing Correlation Relationships: 410752it [00:09, 45097.42it/s]
Adding Correlation Relationships: 100%|██████████| 410752/410752 [55:29<00:00, 123.37it/s]   

Graph database successfully updated.


In [84]:
model = ChatOpenAI(model="gpt-4o")
agent = create_tool_calling_agent(model, [neo4j_tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[neo4j_tool])

Adding Sector Correlation Relationships: 126060it [19:45, 106.37it/s]

Graph creation complete!


In [ ]:
def add_relationships(graph, relationships, start_label, end_label, relationship_type, properties=None):
    """
    Adds relationships between nodes in Neo4j using Cypher.

    Parameters:
        graph: Neo4jGraph instance
        relationships: list of tuples [(start_node_id, end_node_id, ...)]
        start_label: Label for the start node (e.g., 'Stock')
        end_label: Label for the end node (e.g., 'Sector')
        relationship_type: Type of relationship (e.g., 'BELONGS_TO')
        properties: List of property names to include in the relationship
    """
    for rel in tqdm(relationships, desc=f"Adding {relationship_type} Relationships"):
        start_id, end_id, *rel_props = rel
        property_string = ""
        if properties and rel_props:
            # Generate Cypher property string dynamically
            property_string = ", ".join([f"{prop}: {value}" for prop, value in zip(properties, rel_props)])
            property_string = f"{{ {property_string} }}"
        
        query = f"""
        MATCH (start:{start_label} {{name: '{start_id}'}}), (end:{end_label} {{name: '{end_id}'}})
        MERGE (start)-[r:{relationship_type} {property_string}]->(end)
        """
        graph.query(query)

In [ ]:
print("Adding Sector Nodes...")
sectors = stocks_complete["Sector"].unique()
for sector in tqdm(sectors, desc="Adding Sector Nodes"):
    query = f"MERGE (:Sector {{name: '{sector}'}})"
    graph.query(query)

In [ ]:
print("Adding Stock Nodes...")

# Prepare data for batch processing with only key variables
stock_data = [
    {
        "Ticker": row["Ticker"],
        "Sector": row["Sector"],
        "MarketCap": row["marketCap"],  # Key metric for company size
        "TrailingPE": row["trailingPE"],  # Valuation metric
        "RSI": row["RSI"],  # Momentum indicator
    }
    for _, row in stocks_complete.iterrows()
]

batch_size = 100  # Adjust batch size for better performance
batches = [stock_data[i:i + batch_size] for i in range(0, len(stock_data), batch_size)]

for batch in tqdm(batches, desc="Adding Stock Nodes in Batches"):
    query = """
    UNWIND $batch AS row
    MERGE (s:Stock {name: row.Ticker})
    SET s += {
        sector: row.Sector,
        marketCap: row.MarketCap, 
        trailingPE: row.TrailingPE, 
        RSI: row.RSI
    }
    WITH s, row
    MATCH (sec:Sector {name: row.Sector})
    MERGE (s)-[:BELONGS_TO]->(sec)
    """
    graph.query(query, parameters={"batch": batch})

In [ ]:
print("Adding Correlation Relationships...")
correlations = rolling_corr_matrices[rolling_corr_matrices["Correlation"] > 0.8]

# Prepare data for relationships
correlation_data = [
    {
        "Ticker1": row["Ticker1"],
        "Ticker2": row["Ticker2"],
        "Correlation": row["Correlation"]
    }
    for _, row in correlations.iterrows()
]

batch_size = 100  # Adjust batch size
correlation_batches = [
    correlation_data[i:i + batch_size]
    for i in range(0, len(correlation_data), batch_size)
]

for batch in tqdm(correlation_batches, desc="Adding Correlation Relationships"):
    query = """
    UNWIND $relationships AS rel
    MATCH (s1:Stock {name: rel['Ticker1']}), (s2:Stock {name: rel['Ticker2']})
    MERGE (s1)-[:CORRELATES_WITH {correlation: rel['Correlation']}]->(s2)
    """
    graph.query(query, {"relationships": batch})

In [ ]:
print("Adding Sector Correlation Relationships...")

# Filter relationships by correlation threshold
correlation_threshold = 0.85
filtered_corr = rolling_sector_corr[rolling_sector_corr["Correlation"] >= correlation_threshold]

sector_data = [
    {
        "Sector1": row["Sector1"],
        "Sector2": row["Sector2"],
        "Correlation": row["Correlation"]
    }
    for _, row in filtered_corr.iterrows()
]

batch_size = 100  # Adjust batch size
sector_batches = [
    sector_data[i:i + batch_size]
    for i in range(0, len(sector_data), batch_size)
]

for batch in tqdm(sector_batches, desc="Adding Sector Correlation Relationships"):
    query = """
    UNWIND $relationships AS rel
    MATCH (s1:Sector {name: rel['Sector1']}), (s2:Sector {name: rel['Sector2']})
    MERGE (s1)-[:CORRELATES_WITH {correlation: rel['Correlation']}]->(s2)
    """
    graph.query(query, {"relationships": batch})

In [ ]:
def update_stock_realtime(ticker):
    stock_data = yf.Ticker(ticker).history(period="1d")
    if not stock_data.empty:
        latest_close = stock_data["Close"].iloc[-1]
        query = f"""
        MATCH (s:Stock {{name: '{ticker}'}})
        SET s.latestClose = {latest_close}
        """
        graph.query(query)
        print(f"Updated latestClose for {ticker}.")

update_stock_realtime("AAPL")

In [ ]:
query = "What is the most influential stock in the tech sector?"
response = agent_executor.invoke({"input": query})
print(response)

print("Graph construction and integration complete!")